In [77]:
import requests
import re
import os
from datetime import datetime
from urllib.parse import urljoin, urlparse, parse_qs
from bs4 import BeautifulSoup

# 关键参数（从公告中提取）
project_id = "2305042190"
info_id = "188600059995451"

# 方案1：直接构造PDF URL（成功率60%）
pdf_url = f"https://www.shabidding.com/ekp/resource/pdfjs/web/viewer.html?file=%2Fekp%2Fresource%2Fattachment%2Fsys_att_main%2FsysAttMain.do%3Fmethod%3Ddownload%26fdId%3D197539029e1051cce11abbc44a5be0c3"

# 方案2：通过查看器页面解析（更可靠）
viewer_url = f"https://www.shabidding.com/ekp/resource/build/pdf/web/viewer.html?file={project_id}.pdf"



In [78]:
storethis="https://www.shabidding.com/ekp/resource/attachment/sys_att_main/sysAttMain.do?method=download&fdId=197539029e1051cce11abbc44a5be0c3"







In [79]:
from urllib.parse import unquote, urlparse, parse_qs, urljoin

def extract_pdf_url(viewer_url):
    
    try:
        
        parsed = urlparse(viewer_url)
        query = parse_qs(parsed.query)
        
        
        if 'file' not in query:
            raise ValueError("URL 中缺少 file 参数")
        
        encoded_path = query['file'][0]
        
        # 解码路径
        decoded_path = unquote(encoded_path)
        
        # 构建基础 URL
        base_url = f"{parsed.scheme}://{parsed.netloc}"
        
        # 返回完整 PDF URL
        return urljoin(base_url, decoded_path)
    
    except Exception as e:
        print(f"解析失败: {e}")
        return None

In [80]:
viewer_url="https://www.shabidding.com/ekp/resource/pdfjs/web/viewer.html?file=%2Fekp%2Fresource%2Fattachment%2Fsys_att_main%2FsysAttMain.do%3Fmethod%3Ddownload%26fdId%3D197539029e1051cce11abbc44a5be0c3"
pdf_url = extract_pdf_url(viewer_url)
print(pdf_url)


https://www.shabidding.com/ekp/resource/attachment/sys_att_main/sysAttMain.do?method=download&fdId=197539029e1051cce11abbc44a5be0c3


In [81]:
import os
import requests
import pdfplumber
from urllib.parse import urlparse

def download_pdf(url, save_path=None):
    """
    从URL下载PDF文件
    :param url: PDF文件的URL地址
    :param save_path: 可选的自定义保存路径
    :return: 下载文件的本地路径
    """
    try:
        response = requests.get(url, stream=True)
        response.raise_for_status()  # 检查HTTP错误

        # 生成一个有意义的PDF文件名
        parsed_url = urlparse(url)
        query_params = parse_qs(parsed_url.query)
        
        # 如果URL中有fdId参数，使用它作为文件名的一部分
        if 'fdId' in query_params:
            filename = f"document_{query_params['fdId'][0]}.pdf"
        else:
            # 如果没有fdId，使用时间戳作为文件名
            from datetime import datetime
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            filename = f"document_{timestamp}.pdf"
        
        # 设置默认保存路径（当前目录）
        if not save_path:
            save_path = os.path.join(os.getcwd(), filename)
        else:
            save_path = os.path.join(save_path, filename)
        
        # 保存PDF文件
        with open(save_path, 'wb') as f:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)
                
        print(f"PDF下载成功: {save_path}")
        return save_path
        
    except Exception as e:
        print(f"下载失败: {e}")
        return None

def extract_text_from_pdf(pdf_path):
    """
    从PDF文件中提取文本
    :param pdf_path: PDF文件本地路径
    :return: 提取的文本内容
    """
    try:
        import PyPDF2
        
        full_text = []
        with open(pdf_path, 'rb') as file:
            # 创建 PDF 读取器对象
            pdf_reader = PyPDF2.PdfReader(file)
            
            # 遍历每一页
            for page in pdf_reader.pages:
                text = page.extract_text()
                if text:
                    full_text.append(text)
        
        # 合并所有文本
        text = '\n'.join(full_text)
        
        # 如果使用 PyPDF2 提取失败，尝试使用 pdfplumber
        if not text or text.count('cid:') > len(text) / 10:  # 如果CID太多，认为提取失败
            print("PyPDF2提取失败，尝试使用pdfplumber...")
            with pdfplumber.open(pdf_path) as pdf:
                full_text = []
                for page in pdf.pages:
                    text = page.extract_text()
                    if text:
                        full_text.append(text)
                text = '\n'.join(full_text)
        
        return text
    
    except Exception as e:
        print(f"文本提取失败: {e}")
        return None

def save_text_to_file(text, output_path=None):
    """
    将文本保存到文件
    :param text: 要保存的文本内容
    :param output_path: 可选的自定义输出路径
    """
    if not text:
        print("无文本内容可保存")
        return False
    
    # 设置默认输出文件名
    if not output_path:
        output_path = "extracted_text.txt"
    
    try:
        with open(output_path, 'w', encoding='utf-8') as f:
            f.write(text)
        print(f"文本已保存至: {output_path}")
        return True
    except Exception as e:
        print(f"保存失败: {e}")
        return False

def pdf_to_text_pipeline(pdf_url, output_dir=None):
    """
    完整的PDF处理Pipeline
    :param pdf_url: PDF文件的URL
    :param output_dir: 可选的自定义输出目录
    """
    # 创建输出目录（如果不存在）
    if output_dir and not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # 步骤1: 下载PDF
    pdf_path = download_pdf(pdf_url, save_path=output_dir)
    if not pdf_path:
        return False
    
    # 步骤2: 提取文本
    extracted_text = extract_text_from_pdf(pdf_path)
    if not extracted_text:
        return False
    
    # 步骤3: 保存文本
    if output_dir:
        txt_filename = os.path.splitext(os.path.basename(pdf_path))[0] + ".txt"
        output_path = os.path.join(output_dir, txt_filename)
    else:
        output_path = None
    
    return save_text_to_file(extracted_text, output_path)

In [82]:
download_pdf(pdf_url)

PDF下载成功: d:\zhuomian\fenxi\document_197539029e1051cce11abbc44a5be0c3.pdf


'd:\\zhuomian\\fenxi\\document_197539029e1051cce11abbc44a5be0c3.pdf'

In [83]:
# Try extracting text with the improved function
pdf_path = "document_197539029e1051cce11abbc44a5be0c3.pdf"
extracted_text = extract_text_from_pdf(pdf_path)

# Print the first 500 characters to check the result
print("Extracted text preview:")
print(extracted_text[:500] if extracted_text else "No text extracted")


Extracted text preview:
/0/1/2/3/4/5/6/7/8/9/10/11/12/13/14/15/16/17/18/19/20/21/22/23/24/25/23/24/26/27/28
/29 /30 /31 /32 /31 /33 /29 /34 /35 /31 /36/21/22/37/38/i255
/40/41/42 /43/44/45 /i255 /45/47/48/49/50/51/52 /i255 /53/54/55 /i255 /56 /57 /56 /58 /59 /57 /60 /59 /57 /61/i255 /62/63 /i255 /64/65/66/67
/68 /69/70/71 /72 /73 /72 /74 /75 /73 /76 /75 /73 /77 /78/79/80 /81/82 /83 /83 /76 /76 /77 /73 /74 /77 /77 /77 /74 /76 /84 /83 /82
/21 /22/85/86 /87/88/89/90/91 /92/93/89/90 /78/79/85/86 /89/90/66/67
/21/22/5 /94/9


In [84]:
# Try saving in the current working directory first
current_dir = os.getcwd()
print(f"Current working directory: {current_dir}")

# Extract text from PDF
pdf_path = "document_197539029e1051cce11abbc44a5be0c3.pdf"
extracted_text = extract_text_from_pdf(pdf_path)

# Save in the current directory
output_file_path = os.path.join(current_dir, "extracted_text.txt")
save_text_to_file(extracted_text, output_path=output_file_path)


Current working directory: d:\zhuomian\fenxi
文本已保存至: d:\zhuomian\fenxi\extracted_text.txt


True